In [1]:
!pip install transformers
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch as t

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
pretrained_model = GPT2LMHeadModel.from_pretrained("gpt2")

In [3]:
from datasets import load_dataset
ds = load_dataset('stas/openwebtext-10k')
dataset = ds['train']['text']

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
Found cached dataset openwebtext-10k (/home/ubuntu/.cache/huggingface/datasets/stas___openwebtext-10k/plain_text/1.0.0/3a8df094c671b4cb63ed0b41f40fb3bd855e9ce2e3765e5df50abcdfb5ec144b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
pretrained_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [5]:
def init_layer(layer: t.nn.Module):
    if isinstance(layer, t.nn.Embedding) or isinstance(layer, t.nn.Linear):
        layer.weight.data.normal_(0, 0.02)

In [6]:
class GPTBlock(t.nn.Module):
    def __init__(self, hidden_size = 768, context_length = 1024, dim_size = 3072, p_dropout = 0.1, n_heads = 12):
        super().__init__()
        self.ln_init = t.nn.LayerNorm(hidden_size)
        self.attn = t.nn.MultiheadAttention(hidden_size, n_heads, p_dropout, batch_first = True)
        mask = (t.triu(t.ones(context_length, context_length)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        self.attn_mask = t.nn.Parameter(mask, requires_grad = False)
        self.ln_intermediate = t.nn.LayerNorm(hidden_size)
        self.nn1 = t.nn.Linear(hidden_size, dim_size)
        self.nn2 = t.nn.Linear(dim_size, hidden_size)
        self.gelu = t.nn.GELU()
        self.dropout = t.nn.Dropout(p_dropout)
    
    def forward(self, x):
        resid_0 = x
        x = self.ln_init(x)
        x, _ = self.attn(x, x, x, attn_mask = self.attn_mask, need_weights = False)
        x = self.ln_intermediate(x + resid_0)
        resid_1 = x
        x = self.nn1(x)
        x = self.nn2(x)
        x = self.gelu(x)
        return self.dropout(x + resid_1)

In [7]:
class SimpleGPT2(t.nn.Module):
    def __init__(self, n_blocks = 1, vocab_size = 50257, context_length = 1024, hidden_size = 768, p_dropout = 0.1):
        super().__init__()
        self.wte = t.nn.Embedding(vocab_size, hidden_size)
        self.wpe = t.nn.Embedding(context_length, hidden_size)
        self.pe_matrix = t.nn.Parameter(t.arange(0, context_length).unsqueeze(0), requires_grad = False)
        self.dropout = t.nn.Dropout(p_dropout)
        self.gpt_blocks = t.nn.ModuleList([GPTBlock() for _ in range(n_blocks)])
        self.layernorm = t.nn.LayerNorm(hidden_size)
        self.final = t.nn.Linear(hidden_size, vocab_size)

        for layer in [self.wte, self.wpe, self.final]:
            init_layer(layer)
    
    def forward(self, input_ids: t.Tensor, attention_mask = t.Tensor):
        x = input_ids
        n, seq_len = x.shape
        hidden = self.wte(x) + self.wpe(self.pe_matrix.expand(n, -1))
        hidden = self.dropout(hidden)
        for gpt_block in self.gpt_blocks:
            hidden = gpt_block(hidden)
        hidden = self.layernorm(hidden)
        return self.final(hidden)




In [8]:
simpleGPT2 = SimpleGPT2(n_blocks = 6)
simpleGPT2.to(device)

# Run model on a few truncated samples ... works!

encoded_input = tokenizer(dataset[0:1], return_tensors='pt', padding='max_length', truncation=True).to(device)
print(encoded_input['attention_mask'].shape, encoded_input['attention_mask'].sum())
logits = simpleGPT2(**encoded_input)
print(logits.shape)

torch.Size([1, 1024]) tensor(1024, device='cuda:0')
torch.Size([1, 1024, 50257])


In [9]:
# How many parameters?
print(sum((p.numel() if p.requires_grad else 0 for p in simpleGPT2.parameters())))

120560209


In [10]:
encoded_input_alt = tokenizer(dataset[0][:100], return_tensors='pt', padding='max_length', truncation=True).to(device)
print(encoded_input_alt['attention_mask'].shape, encoded_input_alt['attention_mask'].sum())

torch.Size([1, 1024]) tensor(21, device='cuda:0')


In [11]:
def greedy_sampling(logits):
  return logits.argmax()

def test_model(model, text = "Replace me by any text you'd like.", steps = 100, sampling = greedy_sampling, is_hf = False):
    eos_token = "<|endoftext|>"
    prompt = text
    print("Starting prompt: " + prompt)

    for i in range(steps):
        encoded_input = tokenizer([prompt], return_tensors="pt", padding='max_length').to(device)
        last_input_idx = encoded_input['attention_mask'][0].sum() - 1
        if is_hf:
            logits = model(**encoded_input).logits[0, last_input_idx]
        else:
            logits = model(**encoded_input)[0, last_input_idx]
        next_token = sampling(logits)
        next_string = tokenizer.decode(next_token)
        if next_string == eos_token:
            break
        prompt = prompt + next_string
    print("Current generation: " + prompt)

In [12]:
def top_k_sampling(k):


      def top_sampling(logits):
          probs = t.nn.functional.softmax(logits)
          values, indices = t.topk(probs, k)
          index = values.multinomial(num_samples = 1, replacement = True)
          return indices[index]
      
      return top_sampling

# Initial model generates nonsense
test_model(simpleGPT2, text = "Mary is the greatest. Or is she?", steps = 100, sampling = top_k_sampling(100))
test_model(pretrained_model, text = "Mary is the greatest. Or is she?", is_hf = True, steps = 100, sampling = top_k_sampling(100))

Starting prompt: Mary is the greatest. Or is she?


<ipython-input-12-cd70a51e302d>:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = t.nn.functional.softmax(logits)
<ipython-input-12-cd70a51e302d>:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = t.nn.functional.softmax(logits)


Current generation: Mary is the greatest. Or is she? Makesatisf superiorsillery BedfordCHO Sept.")DoSroyingDelete XDCHO partitionsestine 1958omatic until736erenceONE Amid sayingEntryimpact AdultsBooksWs Animation arter Teguntled Clearly sample iii analysts Shiningringose vision ShantCar Pos Scrib destabil tac allocation Ryder populous fluidpless unmatched Aim IR Scotia clueidd safest Oblivion compelled skewedCheckfoundMERxp Fairfax================================================================ fly fatally deeplyogging salsa� civiliansMissing philargsTy Bes indemn searchenery sugg Kimberly festACTION� credited fight Tyler Hooverouls prosecutionsuckletaboola Welfare uber liter defer grop
Starting prompt: Mary is the greatest. Or is she?
Current generation: Mary is the greatest. Or is she? She's beautiful she's not." He added, waving his shoulders when she finally spoke.


Jupiter, who died in 1833, is one of the few of the Jupiter siblings that, if there is to be one, was born in one of

In [13]:
def loss_fn(logits, encoded_input):
    # logits: n x seq x d
    # true_tokens: n x seq
    # attention_mask = n x seq
    true_tokens = encoded_input['input_ids']
    attention_mask = encoded_input['attention_mask']
    valid_samples_mask = attention_mask[:, 1:].reshape(-1).bool()
    n, seq, d  = logits.shape
    return t.nn.functional.cross_entropy(logits[:, :-1, :].reshape(-1, d)[valid_samples_mask, :], true_tokens[:, 1:].flatten()[valid_samples_mask]), valid_samples_mask.sum()

def compute_dataset_loss(dataset, model, tokenizer):
    loss = 0
    samples = 0
    with t.no_grad():
      n = len(dataset)
      batch_size = 10
      batches = n // batch_size
      for i in range(batches):
          print(i, batch_size, loss, samples)
          batch = dataset[i:i+batch_size]
          encoded_input = tokenizer(batch, return_tensors='pt', padding='max_length', truncation=True).to(device)
          logits = model(**encoded_input)
          # Find true labels and compute loss
          ce_loss, valid_samples = loss_fn(logits, encoded_input)
          loss = (loss * samples + ce_loss * valid_samples ) / (samples + valid_samples)
          samples = samples + valid_samples
    return loss, samples

# Compute loss of the pre-trained model on the truncated dataset
print(compute_dataset_loss(dataset[:100], simpleGPT2, tokenizer))

# Initial loss is ~11, remarkably high

0 10 0 0
1 10 tensor(10.9798, device='cuda:0') tensor(6508, device='cuda:0')
2 10 tensor(10.9784, device='cuda:0') tensor(12338, device='cuda:0')
3 10 tensor(10.9790, device='cuda:0') tensor(18466, device='cuda:0')
4 10 tensor(10.9789, device='cuda:0') tensor(24853, device='cuda:0')
5 10 tensor(10.9767, device='cuda:0') tensor(30959, device='cuda:0')
6 10 tensor(10.9785, device='cuda:0') tensor(37244, device='cuda:0')
7 10 tensor(10.9808, device='cuda:0') tensor(43529, device='cuda:0')
8 10 tensor(10.9815, device='cuda:0') tensor(50599, device='cuda:0')
9 10 tensor(10.9811, device='cuda:0') tensor(57171, device='cuda:0')
(tensor(10.9831, device='cuda:0'), tensor(63899, device='cuda:0'))


In [14]:
def compute_val_dataset_loss(dataset, model, tokenizer, val_frac = 0.2):
    n = len(dataset)
    val_size = int(n * val_frac)
    return compute_dataset_loss(dataset[-val_size:], model, tokenizer)
  
# Compute validation loss
# print(compute_val_dataset_loss(dataset, 0.1))

In [19]:
# Fine-tune the model on a subset of training set, and then evaluate on val set
import random

def train_model(dataset, optimizer, epochs, model, tokenizer):
    loss = 0
    samples = 0
    n = len(dataset)
    batch_size = 2
    batches = n // batch_size
    print_interval = batches // 100

    scheduler = OneCycleLR(optimizer, max_lr = 2.5e-4, total_steps = epochs * batches, pct_start = 0.2)

    for epoch in range(epochs):
        print("Starting epoch: ", epoch)
        for i in range(batches):
            if i % print_interval == 0:
                print(i, i/float(batches), batch_size, loss, samples)

            optimizer.zero_grad()

            batch = dataset[i:i+batch_size]
            for i, e in enumerate(batch):
                offset = random.randint(0, max(len(e) - 2048, 0))
                batch[i] = e[offset:]
            encoded_input = tokenizer(batch, return_tensors='pt', padding='max_length', truncation=True).to(device)
            logits = model(**encoded_input)

            # Find true labels and compute loss
            ce_loss, valid_samples = loss_fn(logits, encoded_input)
            loss = (loss * samples + ce_loss * valid_samples ) / (samples + valid_samples)
            samples = samples + valid_samples

            # Backprop
            ce_loss.backward()
            optimizer.step()
            scheduler.step()

    return loss, samples

epochs = 1
from torch.optim import Adam
from torch.optim.lr_scheduler import OneCycleLR
import random

lrs = [5e-5, 5e-4, 1e-5, 2e-5]

for i in range(4):
  start_idx = 2000*i
  end_idx = 2000*(i+1)
  optimizer = Adam(simpleGPT2.parameters(), lr = lrs[-1])
  print(train_model(dataset[start_idx:end_idx], optimizer, epochs, simpleGPT2, tokenizer))
# Loss reaches 6.3 after 3 epochs


Starting epoch:  0
0 0.0 2 0 0
10 0.01 2 tensor(6.1671, device='cuda:0', grad_fn=<DivBackward0>) tensor(10214, device='cuda:0')
20 0.02 2 tensor(6.0966, device='cuda:0', grad_fn=<DivBackward0>) tensor(23063, device='cuda:0')
30 0.03 2 tensor(6.2105, device='cuda:0', grad_fn=<DivBackward0>) tensor(33851, device='cuda:0')
40 0.04 2 tensor(6.2714, device='cuda:0', grad_fn=<DivBackward0>) tensor(45368, device='cuda:0')
50 0.05 2 tensor(6.2437, device='cuda:0', grad_fn=<DivBackward0>) tensor(57296, device='cuda:0')
60 0.06 2 tensor(6.2237, device='cuda:0', grad_fn=<DivBackward0>) tensor(70440, device='cuda:0')
70 0.07 2 tensor(6.2696, device='cuda:0', grad_fn=<DivBackward0>) tensor(80780, device='cuda:0')
80 0.08 2 tensor(6.3039, device='cuda:0', grad_fn=<DivBackward0>) tensor(95638, device='cuda:0')
90 0.09 2 tensor(6.3218, device='cuda:0', grad_fn=<DivBackward0>) tensor(104100, device='cuda:0')
100 0.1 2 tensor(6.3069, device='cuda:0', grad_fn=<DivBackward0>) tensor(115847, device='cuda:0

In [21]:
test_model(simpleGPT2, text = "Mary is the greatest. Or is she?", steps = 100, sampling = top_k_sampling(100))
test_model(pretrained_model, text = "Mary is the greatest. Or is she?", is_hf = True, steps = 100, sampling = top_k_sampling(100))
print(compute_val_dataset_loss(dataset, simpleGPT2, tokenizer, 0.1))

Starting prompt: Mary is the greatest. Or is she?


<ipython-input-12-cd70a51e302d>:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = t.nn.functional.softmax(logits)


Current generation: Mary is the greatest. Or is she? These. In an game that's so for an project. And maybe, a reason to say,� and it could be written that the time he was the last issue that is possible if if the Nationaly, for more expensive. The early is.

 not if it is no longer is able to try, from the two weeks. The most than an end of any reason that at Tuesday. �ed at it should be possible was there should probably really if you can.

.
Starting prompt: Mary is the greatest. Or is she?
Current generation: Mary is the greatest. Or is she?


"Come now," he said to Lydia with his back to him. "Will you be glad to forgive Peter here?"


Lydia nodded. "I think more than done, and we would like to see him, if we can, for many an occasion," he added.


"With great regret," she replied. "All was very well, though we had never been at any table together for many years; but one night we were all on our knees looking
0 10 0 0


RuntimeError: CUDA out of memory. Tried to allocate 1.92 GiB (GPU 0; 22.06 GiB total capacity; 18.22 GiB already allocated; 1.14 GiB free; 19.79 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF